# 0. Libs

In [2]:
from spacy import load # lib para carregar o modelo em português
from spacy.matcher import Matcher # lib para criar regras para detectar palavras positivas e negativas
from spacy.tokens import Doc # lib para armazenar o sentimento em doc 

# 1. Carregar o modelo para o idioma português (pt_core_news_sm)

In [3]:
# !pip install spacy

In [4]:
# !python -m spacy download pt_core_news_sm

In [5]:
nlp = load("pt_core_news_sm") # Modelo "pequeno" de notícias que será usado para análise de sentimentos

# 2. Criar regras com Matcher para detectar palavras positivas e negativas

In [6]:
# Detecção de sentimentos
matcher = Matcher(nlp.vocab)

# Palavras positivas
positive_patterns = [
    [{"LOWER": "bom"}],
    [{"LOWER": "ótimo"}],
    [{"LOWER": "excelente"}],
    [{"LOWER": "adorei"}],
    [{"LOWER": "incrível"}],
    [{"LOWER": "amei"}],
    [{"LOWER": "feliz"}],
    [{"LOWER": "alegre"}],
    [{"LOWER": "maravilhoso"}],
    [{"LOWER": "superou"}, {"LOWER": "expectativas"}],
]
matcher.add("POSITIVE", positive_patterns)

# Palavras negativas
negative_patterns = [
    [{"LOWER": "ruim"}],
    [{"LOWER": "péssimo"}],
    [{"LOWER": "detestei"}],
    [{"LOWER": "odeio"}],
    [{"LOWER": "triste"}],
    [{"LOWER": "decepcionante"}],
    [{"LOWER": "horrível"}],
    [{"LOWER": "lento"}],
    [{"LOWER": "bug"}],
    [{"LOWER": "arrependi"}],
]
matcher.add("NEGATIVE", negative_patterns)

# 3. Adicionar uma extensão personalizada ao objeto Doc para armazenar o sentimento detectado

In [7]:
# Criação de doc para armazenar os sentimentos
Doc.set_extension("sentiment", default=None, force=True)

# Função para analisar o sentimento de um Doc
def analyze_sentiment(doc):
    positive_matches = 0
    negative_matches = 0
    matches = matcher(doc)
    for match_id, start, end in matches:
        if nlp.vocab.strings[match_id] == "POSITIVE":
            positive_matches += 1
        elif nlp.vocab.strings[match_id] == "NEGATIVE":
            negative_matches += 1

    if positive_matches > negative_matches:
        doc._.sentiment = "Positivo"
    elif negative_matches > positive_matches:
        doc._.sentiment = "Negativo"
    elif positive_matches > 0 or negative_matches > 0:
        doc._.sentiment = "Neutro/Misto"
    else:
        doc._.sentiment = "Não detectado"
    return doc._.sentiment

# 4. Analisar uma lista com pelo menos 5 frases e imprimir o sentimento de cada uma

In [8]:
frases = [
    "A comida estava muito boa e o atendimento foi excelente.",
    "Não gostei nada do filme, achei péssimo e lento.",
    "O produto chegou rápido, mas a embalagem estava um pouco amassada.",
    "Adorei a nova atualização do sistema, tudo ficou mais fluido.",
    "Me arrependi da compra, o aparelho apresentou um bug constante.",
    "O dia hoje está maravilhoso e ensolarado.",
    "Achei o preço um pouco alto, mas o produto cumpre a função regular.",
]

print("Análise de Sentimento das Frases:")
for frase in frases:
    doc = nlp(frase)
    sentimento = analyze_sentiment(doc)
    print(f"Frase: '{frase}' - Sentimento: {sentimento}")

Análise de Sentimento das Frases:
Frase: 'A comida estava muito boa e o atendimento foi excelente.' - Sentimento: Positivo
Frase: 'Não gostei nada do filme, achei péssimo e lento.' - Sentimento: Negativo
Frase: 'O produto chegou rápido, mas a embalagem estava um pouco amassada.' - Sentimento: Não detectado
Frase: 'Adorei a nova atualização do sistema, tudo ficou mais fluido.' - Sentimento: Positivo
Frase: 'Me arrependi da compra, o aparelho apresentou um bug constante.' - Sentimento: Negativo
Frase: 'O dia hoje está maravilhoso e ensolarado.' - Sentimento: Positivo
Frase: 'Achei o preço um pouco alto, mas o produto cumpre a função regular.' - Sentimento: Não detectado


# 5. Criar um README.md (conteúdo a ser escrito no arquivo README.md)

In [1]:
readme_content = """
# Análise de Sentimento com spaCy e Regras Personalizadas

Este script em Python utiliza a biblioteca spaCy para realizar análise de sentimentos em frases em português. Ele emprega regras personalizadas definidas com o `Matcher` para identificar palavras e expressões que indicam sentimentos positivos ou negativos. Uma extensão personalizada é adicionada ao objeto `Doc` do spaCy para armazenar o sentimento detectado em cada frase.

## Funcionamento do Código

1.  **Instalação e Carregamento:**
    * O script primeiro carrega o modelo de linguagem português `pt_core_news_sm` do spaCy. Certifique-se de que o spaCy esteja instalado (`pip install spacy`) e o modelo baixado (`python -m spacy download pt_core_news_sm`).

2.  **Definição de Regras:**
    * Utiliza o `Matcher` do spaCy para criar regras de detecção de palavras e expressões.
    * São definidas listas de palavras e frases consideradas positivas (`positive_patterns`) e negativas (`negative_patterns`).
    * Essas regras são adicionadas ao `Matcher` com os IDs "POSITIVE" e "NEGATIVE", respectivamente.

3.  **Extensão Personalizada `Doc.sentiment`:**
    * Uma extensão personalizada chamada `sentiment` é adicionada ao objeto `Doc` do spaCy usando `Doc.set_extension()`. Essa extensão armazenará o sentimento detectado para cada documento (frase processada). O valor padrão é `None`.

4.  **Função `analyze_sentiment(doc)`:**
    * Esta função recebe um objeto `Doc` do spaCy como entrada.
    * Ela aplica o `Matcher` ao documento para encontrar ocorrências das palavras e expressões definidas nas regras positivas e negativas.
    * A função conta o número de correspondências positivas e negativas.
    * Baseado na contagem, o sentimento do documento é determinado:
        * "Positivo" se houver mais correspondências positivas.
        * "Negativo" se houver mais correspondências negativas.
        * "Neutro/Misto" se houver correspondências positivas e/ou negativas, mas sem uma clara predominância.
        * "Não detectado" se nenhuma palavra ou expressão de sentimento for encontrada.
    * O sentimento detectado é armazenado na extensão `doc._.sentiment`.

5.  **Análise de Frases:**
    * Uma lista de frases em português é definida.
    * O script itera sobre essa lista, processando cada frase com o modelo `nlp` para criar um objeto `Doc`.
    * A função `analyze_sentiment()` é chamada para cada `Doc` para determinar seu sentimento.
    * O script imprime a frase original e o sentimento detectado.

## Integrantes

* **Nome:** Danilo Ramalho Silva
* **RM:**   555183

* **Nome:** Israel Dalcin Alves Diniz
* **RM:**   554668

* **Nome:** João Vitor Pires da Silva
* **RM:**   556213

* **Nome:** Matheus Hungaro Fidelis
* **RM:**   555677

* **Nome:** Pablo Menezes Barreto
* **RM:**   556389

* **Nome:** Tiago Toshio Kumagai Gibo
* **RM:**   556984
"""

# Para salvar o README.md em um arquivo:
with open("README.md", "w", encoding="utf-8") as f:
    f.write(readme_content)

print("\nArquivo README.md criado com sucesso!")


Arquivo README.md criado com sucesso!
